In [1]:
from sporgboost.forests import *
from sklearn.metrics import roc_auc_score
from get_data import load
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
from sporgboost.preprocessing import onehot_encode
X, y = load_iris(return_X_y=True)

y = onehot_encode(y)

In [6]:
from sporgboost.common import *
from sporgboost._arrays import *
from sporgboost.projections import *

def _grow_tree(X, y, proj, max_depth, *args):
    # Each piece of work contains a pointer to 
    # the node being processed and the index positions
    # of obs at that node
    node_train_idx = {1 : np.arange(0, X.shape[0])}
    
    # To track the information for the nodes and use numba
    # we need to keep homogenous data types, use dicts
    node_value = {}
    node_split = {}
    node_proj = {}

    depth = 0
    max_depth = np.inf if max_depth is None else max_depth

    while (depth <= max_depth) and len(node_train_idx) > 0:
        # Parallel loop over all nodes to be processed
        for _ in range(len(node_train_idx)):
            # Get node and asociated obs
            node_idx, idx = node_train_idx.popitem()

            X_, y_ = X[idx, :], y[idx, :]

            # Step 1: Check if node is a leaf
            node_value[node_idx] = row_mean(y_).reshape((1, -1))

            # Leaf check 1: at max depth
            if depth == max_depth:
                continue

            # Leaf check 2: partition is pure
            if gini_impurity(node_value[node_idx]) == 0.:
                continue

            # Step 2: If node is not a leaf, find a split
            # Project data based on function
            A = proj(X_, *args)
            print(A)
            # X_proj = X_ @ A

            # # Leaf check 3: partition has no unique levels in X, can't
            # # be partitioned further to improve performance
            # if np.all(row_nunique(X_proj) <= 1):
            #     continue

            # # Evaluate each col and candidate split
            # col, node_split[node_idx] = best_split(X_proj, y_)
            # node_proj[node_idx] = np.ascontiguousarray(A[:, col]).reshape((-1, 1))

            # # Initalize children and add to the next iteration to be processed
            # node_train_idx[node_idx * 10] = idx[(X_proj[:, col] <= node_split[node_idx])]
            # node_train_idx[node_idx * 10 + 1] = idx[(X_proj[:, col] > node_split[node_idx])]
        depth += 1

    return node_value, node_split, node_proj

_grow_tree(X, y, identity, 2)

In [3]:
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# sk = DecisionTreeClassifier()
# sk.fit(X, y)
# plot_tree(sk)

In [4]:
from sporgboost.common import _grow_tree
from sporgboost.projections import *
_grow_tree(X, y, identity, 2)

In [3]:
from sporgboost.forests import *
from sporgboost.trees import AxisAlignedDecisionTree
aa = AxisAlignedDecisionTree(max_depth = 10)
rf = RandomForest()

In [4]:
aa.fit(X, y)

In [ ]:
aa.predict(X)

In [8]:
rf.forest[0].predict(X)

In [5]:
rf.predict_proba(X)

In [19]:
models = {
    # 'rf' : RandomForest(),
    # 'ab' : AdaBoost(),
    'sporf' : SPORF(d=2, s=3),
    'sporgboost' : SPORGBoost(d=2, s=3, max_depth=2),
    # 'rrf' : RotationalRandomForest(K=2),
    # 'rotboost' : RotBoost(K=2)
}

for k, m in models.items():
    print(f"Fitting {k}")
    m.fit(X_train, y_train)


Fitting sporf
Fitting sporgboost


In [22]:
%time
models['sporgboost'].fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

CPU times: total: 594 ms
Wall time: 597 ms


RandomForestClassifier()

In [13]:
rf.predict_proba(X_test)

[array([[0.31, 0.69],
        [0.21, 0.79],
        [0.37, 0.63],
        ...,
        [0.33, 0.67],
        [0.32, 0.68],
        [0.51, 0.49]]),
 array([[0.69, 0.31],
        [0.79, 0.21],
        [0.63, 0.37],
        ...,
        [0.67, 0.33],
        [0.68, 0.32],
        [0.49, 0.51]])]

In [14]:
roc_auc_score(y_test, rf.predict_proba(X_test)[1], multi_class='ovo')

0.9192514784504009

In [24]:
from sklearn.metrics import accuracy_score
# accuracy_score(y_test, rf.predict(X_test))

In [26]:
# Get AUC scores
auc = {key : accuracy_score(y_test, model.predict(X_test)) for key, model in models.items() if key in {'sporf','sporgboost'}}
auc

{'sporf': 0.7630681818181818, 'sporgboost': 0.6098863636363636}